In [18]:
import pandas as pd
import numpy as np

# ==========================================================
# 0. STAGE RAW EVENTS
# ==========================================================
stg_raw_events = df_clean.copy()

# Define marketing columns
utm_cols = ["utm_source", "utm_medium", "utm_campaign", "utm_content", "utm_term"]
click_cols = ["google_click_id", "facebook_click_id",
              "microsoft_click_id", "google_wbraid", "google_gbraid"]

# Standardize dtypes for UTM and click columns to avoid merge errors
for col in utm_cols + click_cols:
    if col in stg_raw_events.columns:
        stg_raw_events[col] = stg_raw_events[col].astype("string")

print("=== STG_RAW_EVENTS (preview) ===")
print(stg_raw_events.head())
stg_raw_events.info()


import pandas as pd
import numpy as np

# ==========================================================
# 0. STAGE RAW EVENTS
# ==========================================================
stg_raw_events = df_clean.copy()

# Marketing columns explicitly
utm_cols = ["utm_source", "utm_medium", "utm_campaign", "utm_content", "utm_term"]
click_cols = ["google_click_id", "facebook_click_id",
              "microsoft_click_id", "google_wbraid", "google_gbraid"]

# Standardize dtypes for all UTM and click columns to string
for col in utm_cols + click_cols:
    if col in stg_raw_events.columns:
        stg_raw_events[col] = stg_raw_events[col].astype("string")

print("=== STG_RAW_EVENTS (preview) ===")
print(stg_raw_events.head())
stg_raw_events.info()


# ==========================================================
# 1. SESSIONIZATION + FORWARD-FILL MARKETING TAGS PER CLIENT
# ==========================================================
events = stg_raw_events.sort_values(["client_id", "timestamp"]).copy()

# Forward-fill UTM and click IDs within each client timeline
for col in utm_cols + click_cols:
    if col in events.columns:
        events[col + "_filled"] = (
            events
            .groupby("client_id")[col]
            .ffill()
        )

# Sessionization logic
timeout = pd.Timedelta(minutes=30)
events["prev_ts"] = events.groupby("client_id")["timestamp"].shift(1)
events["gap"] = events["timestamp"] - events["prev_ts"]
events["new_session"] = events["prev_ts"].isna() | (events["gap"] > timeout)

events["session_index"] = events["new_session"].cumsum()
events["session_id"] = "sess_" + events["session_index"].astype(str)

stg_with_sessions = events.drop(columns=["prev_ts", "gap", "new_session"])

print("\n=== STG_WITH_SESSIONS (preview) ===")
print(stg_with_sessions.head())
stg_with_sessions.info()


# ==========================================================
# 2. DIM_USERS
# ==========================================================
dim_users = (
    stg_with_sessions
    .groupby("client_id")
    .agg(
        first_seen=("timestamp", "min"),
        last_seen=("timestamp", "max"),
        total_events=("event_name", "count"),
        total_sessions=("session_id", "nunique"),
        total_revenue=("revenue", "sum")
    )
    .reset_index()
)

print("\n=== DIM_USERS (preview) ===")
print(dim_users.head())
dim_users.info()


# ==========================================================
# 3. FACT_SESSIONS
# ==========================================================
fact_sessions = (
    stg_with_sessions
    .groupby("session_id")
    .agg(
        client_id=("client_id", "first"),
        session_start=("timestamp", "min"),
        session_end=("timestamp", "max"),
        event_count=("event_name", "count"),
        pageview_count=("event_name", lambda x: (x == "page_viewed").sum()),
        order_count=("transaction_id", lambda x: x.notna().nunique()),
        session_revenue=("revenue", "sum"),
        browser=("browser", "first"),
        os=("ua_os", "first"),
        device=("ua_device", "first")
    )
    .reset_index()
)

fact_sessions["session_duration_seconds"] = (
    fact_sessions["session_end"] - fact_sessions["session_start"]
).dt.total_seconds().fillna(0)

print("\n=== FACT_SESSIONS (preview) ===")
print(fact_sessions.head())
fact_sessions.info()


# ==========================================================
# 4. DIM_UTM  (actual UTM combinations seen in raw events)
# ==========================================================
dim_utm = (
    stg_raw_events[utm_cols]
    .dropna(how="all")          # keep only rows with at least one UTM present
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_utm["utm_id"] = dim_utm.index + 1

print("\n=== DIM_UTM (preview) ===")
print(dim_utm.head())
dim_utm.info()


# ==========================================================
# 5. DIM_CLICK_IDS  (actual click ID combinations seen)
# ==========================================================
dim_click_ids = (
    stg_raw_events[click_cols]
    .dropna(how="all")
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_click_ids["click_id_key"] = dim_click_ids.index + 1

print("\n=== DIM_CLICK_IDS (preview) ===")
print(dim_click_ids.head())
dim_click_ids.info()


# ==========================================================
# 6. FACT_ORDERS  (one row per transaction_id)
#    Uses forward-filled UTM/click data per client
# ==========================================================
order_events = events[
    (events["event_name"] == "checkout_completed")
    & (events["transaction_id"].notna())
].copy()

# Aggregate revenue per transaction
order_revenue = (
    order_events
    .groupby("transaction_id")["revenue"]
    .sum()
    .reset_index()
    .rename(columns={"revenue": "order_revenue"})
)

# One row per transaction (first checkout_completed row in time)
fact_orders = (
    order_events
    .sort_values("timestamp")
    .drop_duplicates("transaction_id", keep="first")
    .merge(order_revenue, on="transaction_id", how="left")
)

# Use forward-filled UTM/click for orders where direct values are missing
for col in utm_cols + click_cols:
    filled_col = col + "_filled"
    if filled_col in fact_orders.columns:
        fact_orders[col] = fact_orders[col].fillna(fact_orders[filled_col])

# Attach UTM and click dimension keys
if not dim_utm.empty:
    fact_orders = fact_orders.merge(dim_utm, on=utm_cols, how="left")

if not dim_click_ids.empty:
    fact_orders = fact_orders.merge(dim_click_ids, on=click_cols, how="left")

# Keep a compact set of columns for the orders fact
keep_cols_orders = [
    "transaction_id",
    "client_id",
    "timestamp",
    "event_date",
    "browser",
    "ua_os",
    "ua_device",
    "order_revenue",
] + utm_cols + click_cols + ["utm_id", "click_id_key"]

fact_orders = fact_orders[keep_cols_orders].rename(columns={
    "timestamp": "order_timestamp",
    "event_date": "order_date",
    "ua_os": "os",
    "ua_device": "device"
})

print("\n=== FACT_ORDERS (preview) ===")
print(fact_orders.head())
fact_orders.info()


# ==========================================================
# 7. FACT_ORDER_ITEMS  (line-item grain with marketing info)
# ==========================================================
item_events = events[
    events["transaction_id"].notna()
    & events["items"].notna()
].copy()

rows = []
for _, row in item_events.iterrows():
    items_list = row["items"]
    if not isinstance(items_list, list):
        continue

    base = row.to_dict()
    for item in items_list:
        rec = {
            "transaction_id": base.get("transaction_id"),
            "client_id": base.get("client_id"),
            "timestamp": base.get("timestamp"),
            "event_date": base.get("event_date"),
            "browser": base.get("browser"),
            "os": base.get("ua_os"),
            "device": base.get("ua_device"),
        }
        # forward-filled marketing at this event
        for col in utm_cols + click_cols:
            rec[col] = base.get(col + "_filled")

        # item details
        rec["item_id"] = item.get("item_id")
        rec["item_name"] = item.get("item_name")
        rec["item_variant"] = item.get("item_variant")
        rec["item_quantity"] = item.get("quantity")
        rec["item_price"] = item.get("item_price")
        rows.append(rec)

fact_order_items = pd.DataFrame(rows)

# Line revenue
fact_order_items["line_revenue"] = (
    fact_order_items["item_quantity"].fillna(0)
    * fact_order_items["item_price"].fillna(0)
)

# Attach dim keys; join on the UTM/click combos used above
if not dim_utm.empty:
    fact_order_items = fact_order_items.merge(dim_utm, on=utm_cols, how="left")

if not dim_click_ids.empty:
    fact_order_items = fact_order_items.merge(dim_click_ids, on=click_cols, how="left")

print("\n=== FACT_ORDER_ITEMS (preview) ===")
print(fact_order_items.head())
fact_order_items.info()


# ==========================================================
# 8. DIM_PRODUCTS (from item-level attributes)
# ==========================================================
product_cols = ["item_name", "item_variant", "item_type", "item_size"]

# Some item_type/item_size may not be present in your df; guard safely
existing_product_cols = [c for c in product_cols if c in stg_raw_events.columns]  # from raw
# Better: try to get them from fact_order_items if present there
existing_product_cols = [c for c in product_cols if c in fact_order_items.columns]

dim_products = (
    fact_order_items[existing_product_cols]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_products["product_id"] = dim_products.index + 1

print("\n=== DIM_PRODUCTS (preview) ===")
print(dim_products.head())
dim_products.info()

# Attach product_id back to fact_order_items
if not dim_products.empty:
    fact_order_items = fact_order_items.merge(dim_products, on=existing_product_cols, how="left")


# ==========================================================
# 9. DIM_ITEMS (item_id to product_id mapping)
# ==========================================================
dim_items = (
    fact_order_items[["item_id", "product_id"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

print("\n=== DIM_ITEMS (preview) ===")
print(dim_items.head())
dim_items.info()


# ==========================================================
# FINAL SHAPES
# ==========================================================
print("\n=== FINAL TABLE SHAPES ===")
print("stg_raw_events:", stg_raw_events.shape)
print("stg_with_sessions:", stg_with_sessions.shape)
print("dim_users:", dim_users.shape)
print("fact_sessions:", fact_sessions.shape)
print("dim_utm:", dim_utm.shape)
print("dim_click_ids:", dim_click_ids.shape)
print("dim_products:", dim_products.shape)
print("dim_items:", dim_items.shape)
print("fact_orders:", fact_orders.shape)
print("fact_order_items:", fact_order_items.shape)



=== STG_RAW_EVENTS (preview) ===
                 client_id                                           page_url  \
0  1740268798-lvfV1k5nHCN8  https://puffy.com/blogs/best-sleep/heating-pad...   
1  1740268852-ioJSoDq7hwNm  https://puffy.com/products/puffy-mattress?size...   
2  1740268767-SDGE_Skxg7P7  https://puffy.com/products/puffy-topper?feel=D...   
3  1739894111-Ck3AEc97wn-A  https://puffy.com/blogs/reviews/mattress-in-a-...   
4  1740264185-OFhxMErmemrD                                 https://puffy.com/   

                       referrer                        timestamp   event_name  \
0  https://source-afb1343a.com/ 2025-02-23 00:00:01.750000+00:00  page_viewed   
1                           NaN 2025-02-23 00:00:54.391000+00:00  page_viewed   
2  https://source-e34e0c2e.com/ 2025-02-23 00:01:07.615000+00:00  page_viewed   
3                           NaN 2025-02-23 00:01:08.642000+00:00  page_viewed   
4                           NaN 2025-02-23 00:02:08.395000+00:00  page_view